In [51]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
#sql牛逼
import MySQLdb
#zz牛逼教我json
import json
#你看看你个菜鸡用多久
import datetime
#系统牛逼
import os
#历史记录？
import time
#正则表达式天下第一
import re

In [52]:
def start():
    #万事开头难
    global conn,curs
    conn = MySQLdb.connect("127.0.0.1", "root", "nideshengri", "nidekuzi", charset='utf8' )
    # 使用cursor()方法获取操作游标 
    curs = conn.cursor()

In [53]:
#插入游戏tag
def inserttag(idn,tags):
    for tag in tags.split(','):
        sql="""INSERT INTO `nidekuzi`.`id_tag`(SteamId,Tag) VALUES (%d,"%s")"""%(idn,tag.lstrip())
        #try:
        curs.execute(sql)
        #except:
            #conn.rollback()
        conn.commit()

In [54]:
#遍历并读取所有文件
def TransAndReadFile(Route):
    global files,alldata
    files=[]
    alldata=[]
    for root, dirs, files in os.walk(r'%s'%Route):
        pass
    #拜拜了您内.json   
    for i in range(len(files)):
        files[i]=files[i][:-5]
    #至此,files列表中存放了路径中的所有文件名
    #print(files[0])
    for mtag in files:
        fp=open(r"%s\\%s.json"%(Route,mtag),"rt", encoding='utf-8')
        file=[]
        for items in fp.readlines():
            for item in json.loads(items):
                file.append(item)
        alldata.append(file)
        fp.close()
    #效果拔群
    #至此，alldata列表中的每项都存放了各个文件的所有内容
    #print(alldata[4][7])

In [55]:
#插入游戏的函数
def insertgame(mtag,idn,name,platforms,pic,price,tags):
    if len(price)==0:
        return      #价格为空意味着游戏已经下架
    name=filt_emo(name)     #为什么有人在游戏名里加emoji？
    name=transferContent(name)      #转义真是太难了太难了...
    price[0]=filt(price[0])
    price[-1]=filt(price[-1])
    if float(price[-1][:-1])==0:
        cut='0%'
    else:
        cut=str(int((float(price[-1][:-1])-float(price[0][:-1]))/float(price[-1][:-1])*100))
        cut=cut+'%'
    sql = """INSERT INTO `%s`(`SteamId`, `Name`,`Platforms`, `Pic_url`,`CurrentPrice`, `LowestPrice`, `OriginalPrice` , `Cut`)
             VALUES ('%d','%s','%s','%s','%s','%s','%s','%s')"""%(mtag,idn,name,platforms,pic,filt(price[0]),filt(price[0]),filt(price[-1]),cut)
    #try:
    curs.execute(sql)
    #except:
        #conn.rollback()
    conn.commit()
    inserttag(idn,tags)

In [56]:
#转义NMSL，代码我抄的https://blog.csdn.net/lengye7/article/details/79916685
def transferContent(content):
        if content is None:
            return None
        else:
            string = ""
            for c in content:
                if c == '"':
                    string += '\\\"'
                elif c == "'":
                    string += "\\\'"
                elif c == "\\":
                    string += "\\\\"
                else:
                    string += c
            return string

In [57]:
#求求你们不要在游戏名里加emoji了！！
def filt_emo(name):
    restr=''
    try:
        co = re.compile(u'[\U00010000-\U0010ffff]')
    except re.error:
        co = re.compile(u'[\uD800-\uDBFF][\uDC00-\uDFFF]')
    return co.sub(restr, name)

In [58]:
#金钱格式一定要统一啊....
def filt(price):
    price=re.findall("\d+\\.?\d*",price)
    if len(price)==0:
        return '0.00￥'
    else:
        price=float(price[0])
        return '%.2f￥'%price

In [59]:
#每日更新数据的函数
def updata(mtag,idn,name,platforms,pic,price,tags):
    price[0]=filt(price[0])
    platform=''
    for a in platforms:
        platform+=a
    sql="SELECT * FROM `%s` WHERE SteamId=%d"%(mtag,idn)
    curs.execute(sql)
    olddata=curs.fetchone()
    if olddata==None:#意味着有新游戏或者游戏遗漏，那么插入游戏
        insertgame(mtag,idn,name,platforms,pic,price,tags)
        return
    if price[0] != olddata[-4]:
        recording(mtag,idn,price[0],time)
    if float(price[0][:-1]) <float(olddata[-1][:-1]):
        lowest=price[0]
    else:
        lowest=olddata[-1]
    if float(price[0][:-1])==0:
        cut='0%'
    else:
        cut=str(int((float(olddata[-3][:-1])-float(price[0][:-1]))/float(olddata[-3][:-1])*100))
        cut=cut+'%'
    sql='''UPDATE `%s` SET `Platforms` = '%s', `CurrentPrice` = '%s', `Cut` = '%s', `LowestPrice` = '%s' , `Pic_url` = '%s' WHERE `%s`.`SteamId` = %d'''%(mtag,platform,price[0],cut,lowest,pic,mtag,idn)

    #try:
    curs.execute(sql)
    #except:
        #conn.rollback()
    conn.commit()

In [60]:
#在updata表里留下证据
def recording(mtag,idn,price,time):
    sql = """INSERT INTO `%sUpdata`(`SteamId`, `Price`)
             VALUES ('%d','%s')"""%(mtag,idn,price)
    #try:
    curs.execute(sql)
    #except:
        #conn.rollback()
    conn.commit()

In [61]:
#通用更新函数,没用到这个
#四个参数为表名，id，需要更新的数据，更新数据的字段名
def updatasql(mtag,idn,data,dataname):
    sql='''UPDATE `%s` SET `%s` = '%s' WHERE `%s`.`SteamId` = %d'''%(mtag,dataname,data,mtag,idn)
    #try:
    curs.execute(sql)
    #except:
        #conn.rollback()
    conn.commit()

In [66]:
#每日更新数据
ks=datetime.datetime.now()
start()
#file的第一项是一个字典就行
for item in and_list :
    updata(item['block'],int(item['id']),item['name'],item['platform'],item['picture'][0],item['price'],item['tags'])
js=datetime.datetime.now()
print(js-ks)

NameError: name 'and_list' is not defined

In [63]:
# 关闭连接
conn.close()